<a href="https://colab.research.google.com/github/ptran1203/pytorch-animeGAN/blob/master/notebooks/animeGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive, output
import torch
dv = torch.cuda.get_device_name(0)
print(dv)
assert 'K80' not in dv, 'Too weak'

drive.mount('/content/drive', force_remount=False)
repo = "Pytorch-animeGAN"
%cd "/content"
!rm -rf {repo}
!git clone https://github.com/ptran1203/{repo}
%cd {repo}
output.clear()

In [2]:
import os
import urllib

data_path = 'anime-gan.zip'
dataset_url = 'https://github.com/ptran1203/pytorch-animeGAN/releases/download/v1.0/dataset_v1.zip'

if not os.path.exists("/content/dataset"):
    !wget -O {data_path} {dataset_url}
    !unzip {data_path} -d /content
    !rm {data_path}

    if not os.path.exists("/content/dataset"):
        raise ValueError(f"Download Failed, {data_path}")

output.clear()

In [3]:
working_dir = '/content/drive/MyDrive/animeGAN'
ckp_dir = f'{working_dir}/checkpoints'
save_img_dir = f'{working_dir}/generated_samples'


In [ ]:
!python3 train.py --dataset 'Shinkai'\
                  --batch 6\
                  --debug-samples 0\
                  --init-epochs 10\
                  --checkpoint-dir {ckp_dir}\
                  --save-image-dir {save_img_dir}\
                  --save-interval 1\
                  --gan-loss lsgan\
                  --init-lr 0.0001\
                  --lr-g 0.00002\
                  --lr-d 0.00004\
                  --wadvd 10.0\
                  --wadvg 10.0\
                  --wcon 1.5\
                  --wgra 3.0\
                  --wcol 30.0\
                  --resume GD\
                  --use_sn\

usage: train.py [-h] [--dataset DATASET] [--data-dir DATA_DIR]
                [--epochs EPOCHS] [--init-epochs INIT_EPOCHS]
                [--batch-size BATCH_SIZE] [--checkpoint-dir CHECKPOINT_DIR]
                [--save-image-dir SAVE_IMAGE_DIR] [--gan-loss GAN_LOSS]
                [--resume RESUME] [--use_sn] [--save-interval SAVE_INTERVAL]
                [--debug-samples DEBUG_SAMPLES] [--lr-g LR_G] [--lr-d LR_D]
                [--init-lr INIT_LR] [--wadvg WADVG] [--wadvd WADVD]
                [--wcon WCON] [--wgra WGRA] [--wcol WCOL]
                [--d-layers D_LAYERS] [--d-noise]
train.py: error: unrecognized arguments: --continu G


In [ ]:
from inference import Transformer
transformer = Transformer(ckp_dir + '/generator_Shinkai.pth')

In [ ]:
import cv2
import random
import matplotlib.pyplot as plt
import numpy as np

def random_img(img_dir):
    # p = '/content/test_4.png'
    p = os.path.join(img_dir, random.choice(os.listdir(img_dir)))
    return cv2.imread(p)[:, :, ::-1]

image = random_img('/content/dataset/test/HR_photo')
image = cv2.resize(image, (768, 512))

anime_img = (transformer.transform(image) + 1) / 2

fig = plt.figure(figsize=(18, 25))
fig.add_subplot(1, 2, 1)
plt.imshow(image)
fig.add_subplot(1, 2, 2)
plt.imshow(anime_img[0])
plt.show()

Inference Image

In [ ]:
# !python3 inference_image.py --checkpoint {ckp_dir}\
#                             --src 'https://entertainment.time.com/wp-content/uploads/sites/3/2013/04/ca-05482_r_cmyk.jpg?w=720&h=480&crop=1'\
#                             --dest /content/sample.png

Inference video

In [ ]:
# !python3 inference_video.py --checkpoint {ckp_dir}\
#                             --src /content/test_vid_3.mp4\
#                             --dest /content/test_vid_3_anime.mp4\
#                             --batch-size 2